In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import lightgbm as lgbm
import xgboost

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB

from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge

from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso



from lightgbm import LGBMRegressor


from xgboost import XGBRegressor

In [2]:
X_train = pd.read_csv("./Dataset/CompressedData.csv")
X_train.head()

,Count,Open,High,Low,Close,Volume,VWAP,new_date,Asset_ID,Target
0,-0.420178,-0.231602,-0.231818,-0.231463,-0.231600,-0.163735,-0.231576,2018-01-01,0,0.000148
1,-0.406696,-0.231558,-0.231774,-0.231420,-0.231556,-0.163475,-0.231532,2018-01-02,0,0.000393
2,-0.412850,-0.231527,-0.231744,-0.231389,-0.231525,-0.163609,-0.231501,2018-01-03,0,0.000549
3,-0.411005,-0.231471,-0.231686,-0.231333,-0.231469,-0.163602,-0.231445,2018-01-04,0,0.000006
4,-0.389464,-0.230984,-0.231194,-0.230852,-0.230981,-0.162833,-0.230958,2018-01-05,0,0.005618


In [3]:
%%time

from sklearn.model_selection import cross_validate

def cross_validate_manual(X, y, model):
    cv = cross_validate(
        estimator = model,
        X = X,
        y = y,
        scoring = ["neg_mean_squared_error"],
        cv = 5,
    )
    
    return cv["test_neg_mean_squared_error"].mean()

CPU times: user 5 µs, sys: 6 µs, total: 11 µs
Wall time: 12.4 µs


In [4]:
def train_basic_models(X, y):
    model_lr = LinearRegression(n_jobs = -1)
    mean_lr = cross_validate_manual(X, y, model_lr)
    print("Linear Regression: ", np.abs(10e6*mean_lr))
    
    model_knn = KNeighborsRegressor(n_neighbors = 10)
    mean_knn = cross_validate_manual(X, y, model_knn)
    print("KNN: ", np.abs(10e6*mean_knn))

    model_dt = DecisionTreeRegressor()
    mean_dt = cross_validate_manual(X, y, model_dt)
    print("Decision Tree: ", np.abs(10e6*mean_dt))

    model_rf = RandomForestRegressor(random_state = 42, n_jobs = -1)
    mean_rf = cross_validate_manual(X, y, model_rf)
    print("Random Forest: ", np.abs(10e6*mean_rf))
    
    model_xgb = XGBRegressor()
    mean_xgb = cross_validate_manual(X, y, model_xgb)
    print("XGB: ", np.abs(10e6*mean_xgb))
    
    model_lgbm = LGBMRegressor()
    mean_lgbm = cross_validate_manual(X, y, model_lgbm)
    print("LGBM: ", np.abs(10e6*mean_lgbm))
    
    

In [5]:
y_train = X_train['Target']
X_train.drop(["Target", 'new_date'], axis = 1, inplace = True)

In [6]:
X_train.head()

,Count,Open,High,Low,Close,Volume,VWAP,Asset_ID
0,-0.420178,-0.231602,-0.231818,-0.231463,-0.231600,-0.163735,-0.231576,0
1,-0.406696,-0.231558,-0.231774,-0.231420,-0.231556,-0.163475,-0.231532,0
2,-0.412850,-0.231527,-0.231744,-0.231389,-0.231525,-0.163609,-0.231501,0
3,-0.411005,-0.231471,-0.231686,-0.231333,-0.231469,-0.163602,-0.231445,0
4,-0.389464,-0.230984,-0.231194,-0.230852,-0.230981,-0.162833,-0.230958,0


In [7]:
train_basic_models(X_train,y_train)

Linear Regression:  3.5078033282214753
KNN:  2.5689498017911667
Decision Tree:  17.738522184099605
Random Forest:  4.622394930089359
XGB:  3.122379749785263
LGBM:  3.042412238666025


### Interpretations

* KNN has the lowest MSE. However, we will st|ill select XGBoost and LGBM because they have more hyperparameters which we can tune which might result into lowest test MSE.